In [ ]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import matplotlib.pyplot as plt
import numpy as np
import os


import logging
from fancylog import fancylog
import fancylog as package


import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2


try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl

from fcutils.file_io.io import load_yaml

from utils import start_server, load_params, add_to_params_dict
from utils import print_cnmfe_components, log_cnmfe_components, load_mmap_video_caiman

bpl.output_notebook()
c, dview, n_processes = start_server()

## Load analysis metadata

In [ ]:
#Paths to motion corrected files
fld = 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p2\\19JUN26'
metadata = load_yaml(os.path.join(fld, "01_PARAMS", "analysis_metadata.yml"))
video = os.path.join(fld, metadata[metadata['video_for_cnmfe_fit']])

if not os.path.isfile(video):
    raise FileExistsError(f"The video file speicfied does not exist: {video}")

## CNMF-E params

In [ ]:
# dataset dependent parameters
frate = 10.                       # movie frame rate
decay_time = 2.              # length of a typical transient in seconds


# parameters for source extraction and deconvolution
prms = load_params(fld)
prms['init_params'] = add_to_params_dict(prms['init_params'],  fr=frate, decay_time=decay_time)
opts = params.CNMFParams()

for k,v in prms.items():
    opts.change_params(params_dict=v)     

# Start logging
fancylog.start_logging(os.path.join(fld, "02_LOGS"), package, file_log_level="INFO", variables=[opts], verbose=True,    filename='cnmfe_fit_logs')
logging.info("Starting CNMF-E fit")

# Add analysis metadata to log
logging.info("ANALYSIS METADATA FILE:")
for k,v in metadata.items():
    logging.info(f"{k}:  {v}")
logging.info(f"Video used for CNMF-E fitting: {metadata['video_for_cnmfe_fit']}: {video}")

## Load video and visualise summary frames
use the summary frames to choose values for min_corr and min_pnr values in the parameters

In [ ]:
# Load memmapped video
Yr, dims, T, images = load_mmap_video_caiman(video)

# Show the average background
f, axarr = plt.subplots(ncols=2, nrows=1, figsize=(10, 5))
axarr[0].imshow(images[1000, :, :], cmap="gray")
_ = axarr[1].imshow(np.median(images, axis=0), cmap="gray")

# compute some summary images (correlation and peak to noisae)
cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=opts.init['gSig'][0], swap_dim=False) 

# inspect the summary images and set the parameters [use this to fine min_corr or min_pnr tune params]
inspect_correlation_pnr(cn_filter, pnr)

### Fit CNMF-E
The cn_filter and pnr images that you can inspect with the code in the previous cell can be used to define threshold for fitting and evaluating CNMF-E. Just make sure to add the values to the params file, then restart the kernel and restart from the top. 

In [ ]:
# Update params in case min_corr or min_pnr where changed after inspection
_params = load_params(fld)['init_params']
opts.change_params(params_dict=_params)
logging.info(f"Update init params with param dict: {_params}")


In [ ]:
logging.info("CNMF-E starting fitting")
Ain = None
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
cnm.fit(images)

logging.info("CNMF-E fitting completed")

## Save CNMF_E

In [ ]:
# How many components were found?
print_cnmfe_components(cnm, msg="After fitting")
log_cnmfe_components(cnm,  msg="After fitting")

# Save model for future use
model_path = os.path.join(fld, "cnmfe_fit.hdf5")
cnm.save(model_path)
logging.info(f"Saved CNMF-E model at {model_path}")